In [1]:
import open3d
import argparse
import sys
sys.path.append('.')
import os
import time
import h5py
import datetime
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn.parallel
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, class_names
from pathlib import Path
from tqdm import tqdm
from utils import test, save_checkpoint, select_avaliable, mkdir
import log
from model.pointnet2 import PointNet2ClsMsg
from model.pointnet import PointNetCls, feature_transform_reguliarzer
import colorsys
from clf import parse_args, root
from adv.chamfer import chamfer_non_batch, chamfer_batch

args = parse_args([])
args.pretrain = 'experiment/weights/clf-pointnet-0.89730-0076.pth'
test_data, test_label = load_data(root, train = False)
testDataset = ModelNetDataLoader(test_data, test_label)
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False)

log.debug('Building Model',args.model_name)
if args.model_name == 'pointnet':
    num_class = 40
    model = PointNetCls(num_class,args.feature_transform).cuda()  
else:
    model = PointNet2ClsMsg().cuda()

if args.pretrain is None:
    log.err('No pretrain model')
    exit()

log.debug('Loading pretrain model...')
checkpoint = torch.load(args.pretrain)
model.load_state_dict(checkpoint)
model.eval()
print('Done')

job: clf model_name: pointnet 
pretrain: experiment/weights/clf-pointnet-0.89730-0076.pth 
batch_size: 16 gpu_count: 1 
/media/james/Ubuntu_Data/dataset/ShapeNet/modelnet40_ply_hdf5_2048/ply_data_test0.h5
/media/james/Ubuntu_Data/dataset/ShapeNet/modelnet40_ply_hdf5_2048/ply_data_test1.h5
Building Model pointnet 
Loading pretrain model... 
Done


In [8]:
def num(x):
    return x.detach().cpu().numpy()

def disp_pt_cloud(x,channal_first = True):
    cloud = open3d.geometry.PointCloud()
    cloud.points = open3d.utility.Vector3dVector(x)

    vis = open3d.visualization.VisualizerWithKeyCallback()
    vis.create_window(args.model_name, height=800, width=800, left=200, top=0)
    opt = vis.get_render_option().background_color = np.asarray([0, 0, 0])
    vis.add_geometry(cloud)
    vis.register_key_callback(32, lambda vis: exit())
    vis.run()
    vis.destroy_window()

for pts, gt in testDataLoader:
    gt = gt[:, 0].long().cuda()
    pts.requires_grad = False
    pts = pts.transpose(2, 1).cuda() # make channal first

    output, _ = model(pts)
    original = output.data.max(1)[1]
    target = torch.from_numpy(np.array([1]* args.batch_size)).cuda()
    noise = torch.zeros_like(pts, requires_grad=True).cuda()
    opt = optim.Adam([noise], lr=0.1, betas=(0.9, 0.999))
    mse_fn = nn.MSELoss()
    cross_fn = nn.CrossEntropyLoss()
    
    for i in range(100):        
        opt.zero_grad()
        pts_adv = pts + noise
        pred, _ = model(pts_adv)
        loss_cross = cross_fn(pred, target)
        loss_chamfer = chamfer_batch(pts_adv.transpose(2, 1), pts.transpose(2, 1))
        loss_l2 = mse_fn(pts_adv, pts) 
        loss = loss_cross + loss_chamfer * 0.01 + loss_l2 * 10
        loss.backward()
        opt.step()
            
        output, _ = model(pts_adv)
        adv_chocie = output.data.max(1)[1]
        untar_rate = (adv_chocie == original).cpu().numpy().mean()
        tar_rate = (adv_chocie == target).cpu().numpy().mean()
        print('%6.2f %6.2f %6.2f %6.2f | untarget = %.2f target = %.2f'%
              (num(loss), num(loss_cross),num(loss_chamfer),num(loss_l2), untar_rate, tar_rate))
        del output,adv_chocie,untar_rate,tar_rate
        
    break
# disp_pt_cloud(num(pts.detach().transpose(2, 1)[0]))
disp_pt_cloud(num(pts_adv.detach().transpose(2, 1)[0]))

 15.73  15.73   0.00   0.00 | untarget = 1.00 target = 0.00
 18.45  10.24 818.48   0.00 | untarget = 0.69 target = 0.00
 18.07   7.14 1086.66   0.01 | untarget = 0.12 target = 0.00
 18.09   5.32 1268.03   0.01 | untarget = 0.12 target = 0.06
 17.78   3.27 1437.58   0.01 | untarget = 0.12 target = 0.06
 17.50   1.71 1562.34   0.02 | untarget = 0.00 target = 0.38
 17.94   1.32 1640.60   0.02 | untarget = 0.06 target = 0.44
 18.05   0.75 1704.73   0.03 | untarget = 0.00 target = 0.81
 18.36   0.59 1748.27   0.03 | untarget = 0.00 target = 0.94
 18.41   0.33 1776.30   0.03 | untarget = 0.00 target = 0.94
 18.56   0.28 1792.63   0.04 | untarget = 0.00 target = 1.00
 18.53   0.24 1790.55   0.04 | untarget = 0.00 target = 1.00
 18.37   0.19 1776.84   0.04 | untarget = 0.00 target = 1.00
 18.09   0.16 1749.40   0.04 | untarget = 0.00 target = 1.00
 17.74   0.12 1716.30   0.05 | untarget = 0.00 target = 1.00
 17.30   0.10 1673.04   0.05 | untarget = 0.00 target = 1.00
 16.86   0.11 1626.88   0.

In [9]:
disp_pt_cloud(num(pts_adv.detach().transpose(2, 1)[1]))